In [1]:
import pandas as pd
import os
 
 
data = pd.read_csv('Postive imdb title.csv', encoding='utf-8')
with open('Postive imdb title.txt','a+', encoding='utf-8') as f:
    for line in data.values:
        f.write((str(line[1])+'\n'))


In [2]:
# import and download necessary packages
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')

# start reading the text file
# establish a connection to the external text file 'epl.txt'
infile = open('Postive imdb title.txt', 'r') 

# read all lines in epl.txt
lines = infile.readlines()

# remove \n at the end of each line 
lines = [l.strip() for l in lines]

# remove empty lines
lines = [l for l in lines if l != ""]

# display the number of lines in the text file
# note: each line is treated as a sole document
len(lines)

[nltk_data] Downloading package punkt to /Users/kawanwong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


1423

In [3]:
# import necessary library
from sklearn.feature_extraction.text import CountVectorizer

# convert our text data to a DTM
tf_vectorizer = CountVectorizer(stop_words='english')
dtm_epl = tf_vectorizer.fit_transform(lines)

In [4]:
# convert our DTM to a numpy array and display it
dtm_epl.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [5]:
# import necessary library
from sklearn.decomposition import LatentDirichletAllocation

# train a latent dirichlet allocation model with number of topics = 3
lda = LatentDirichletAllocation(n_components=10, random_state=0)

# fit the dtm into the lda object
lda.fit(dtm_epl)

LatentDirichletAllocation(random_state=0)

In [6]:
# get the topic-word(term) association for the LDA object
topic_word_matrix = lda.components_

# retrieve top n_top_words words for each topic
no_top_words = 10
feature_names = tf_vectorizer.get_feature_names()

# create a dataframe for displaying the results
rows = []

for topic_id, topic in enumerate(topic_word_matrix):
    row = ["Topic #" + str(topic_id) + ":"]
    row += [
        feature_names[i] + "*" + str(np.round(topic[i] / np.sum(topic), 4))
        for i in topic.argsort()[:-no_top_words - 1:-1]
    ]
    rows.append(row)

topic_word_df = pd.DataFrame(rows, columns=['Topic', 'Top 1 Word*Prob', 'Top 2 Word*Prob', \
                                            'Top 3 Word*Prob', 'Top 4 Word*Prob', 'Top 5 Word*Prob', 'Top 6 Word*Prob', 'Top 7 Word*Prob', 'Top 8 Word*Prob', 'Top 9 Word*Prob', 'Top 10 Word*Prob'])

topic_word_df

,Topic,Top 1 Word*Prob,Top 2 Word*Prob,Top 3 Word*Prob,Top 4 Word*Prob,Top 5 Word*Prob,Top 6 Word*Prob,Top 7 Word*Prob,Top 8 Word*Prob,Top 9 Word*Prob,Top 10 Word*Prob
0,Topic #0:,overhyped*0.0688,brilliant*0.0574,loved*0.0528,fantastic*0.0437,really*0.0368,good*0.026,love*0.0254,light*0.0139,way*0.0131,enjoyed*0.0117
1,Topic #1:,great*0.1197,series*0.0819,good*0.0507,hype*0.0369,worth*0.0211,time*0.0159,new*0.0133,best*0.0126,nice*0.0106,web*0.0102
2,Topic #2:,entertaining*0.0473,games*0.0435,hunger*0.036,meets*0.0266,binge*0.0241,worthy*0.0209,exciting*0.0209,battle*0.019,royale*0.0171,bit*0.0153
3,Topic #3:,best*0.1276,amazing*0.1063,series*0.0388,2021*0.0229,netflix*0.0229,outstanding*0.0153,shows*0.0148,time*0.0078,surprise*0.0078,mindblowing*0.0078
4,Topic #4:,original*0.0278,great*0.0273,movie*0.0241,fun*0.0203,good*0.0189,incredible*0.0178,solid*0.0153,story*0.0128,season*0.0128,like*0.0128
5,Topic #5:,excellent*0.0574,awesome*0.0437,nan*0.0368,start*0.0181,ending*0.0178,good*0.0172,korean*0.0152,series*0.014,recommended*0.014,episode*0.0138
6,Topic #6:,masterpiece*0.0856,classic*0.0209,plot*0.0158,end*0.0158,great*0.013,genius*0.0106,thing*0.0102,absolute*0.008,times*0.008,heart*0.008
7,Topic #7:,overrated*0.1511,slow*0.0205,absolutely*0.0205,disturbing*0.0203,review*0.0115,feel*0.0115,didnt*0.0092,strong*0.0092,emotional*0.0092,modern*0.0092
8,Topic #8:,watch*0.1261,korean*0.0557,worth*0.0402,great*0.0322,life*0.0202,overall*0.0162,drama*0.0133,definitely*0.0117,watched*0.0104,series*0.0104
9,Topic #9:,perfect*0.0629,vips*0.0428,seen*0.0328,ive*0.0228,meh*0.0228,average*0.0203,intense*0.0178,1010*0.0151,things*0.0128,people*0.0127


In [7]:
# document-topic matrix
doc_topic_matrix = lda.transform(dtm_epl)

# specify the number of topics we're interested in
no_top_topic = 2
rows = []

# process each document
for doc_id, doc in enumerate(doc_topic_matrix):
    row = ['# %d' % doc_id]
    # get the top n topics
    row += [
        'Topic' + str(i) + "*" + str(np.round(doc[i], 2))
        for i in doc.argsort()[:-no_top_topic - 1:-1]
    ]
    rows.append(row)

doc_topic_df = pd.DataFrame(
    rows, columns=['Document', 'Top1 Topic*Prob', 'Top2 Topic*Prob'])

doc_topic_df

,Document,Top1 Topic*Prob,Top2 Topic*Prob
0,# 0,Topic4*0.62,Topic8*0.22
1,# 1,Topic1*0.52,Topic6*0.28
2,# 2,Topic5*0.55,Topic6*0.05
3,# 3,Topic7*0.52,Topic9*0.28
4,# 4,Topic1*0.82,Topic7*0.02
...,...,...,...
1418,# 1418,Topic6*0.91,Topic0*0.01
1419,# 1419,Topic5*0.55,Topic6*0.05
1420,# 1420,Topic7*0.28,Topic4*0.27
1421,# 1421,Topic0*0.7,Topic1*0.03


In [8]:
# document-topic matrix
doc_topic_matrix = lda.transform(dtm_epl)

# specify the number of topics we're interested in
no_top_topic = 2
rows = []

# process each document
for doc_id, doc in enumerate(doc_topic_matrix):
    row = ['# %d' % doc_id]
    # get the top n topics
    row += [
        'Topic' + str(i)
        for i in doc.argsort()[:-no_top_topic - 1:-1]
    ]
    rows.append(row)

doc_topic_df = pd.DataFrame(
    rows, columns=['Document', 'Top1 Topic*Prob', 'Top2 Topic*Prob'])

doc_topic_df

,Document,Top1 Topic*Prob,Top2 Topic*Prob
0,# 0,Topic4,Topic8
1,# 1,Topic1,Topic6
2,# 2,Topic5,Topic6
3,# 3,Topic7,Topic9
4,# 4,Topic1,Topic7
...,...,...,...
1418,# 1418,Topic6,Topic0
1419,# 1419,Topic5,Topic6
1420,# 1420,Topic7,Topic4
1421,# 1421,Topic0,Topic1


In [10]:
pip install pyLDAvis

  Using cached pyLDAvis-3.3.1-py2.py3-none-any.whl
  Using cached funcy-1.16-py2.py3-none-any.whl (32 kB)
  Using cached sklearn-0.0-py2.py3-none-any.whl
  Using cached gensim-4.1.2-cp38-cp38-macosx_10_9_x86_64.whl (24.0 MB)
  Using cached smart_open-5.2.1-py3-none-any.whl (58 kB)
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pyLDAvis
import pyLDAvis.sklearn

import pyLDAvis.gensim_models

data = pyLDAvis.sklearn.prepare(lda,dtm_epl,tf_vectorizer,mds='mmds')

In [12]:
pyLDAvis.display(data)

/Applications/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
pyLDAvis.save_html(data, 'imdb pos title.html')

/Applications/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
